In [1]:
import json, cv2,os
import pandas as pd 
from google.colab.patches import cv2_imshow

### change path below to your gdrive path where folder is 

In [2]:
%cd '/content/drive/MyDrive/dev/aiac/AIAC-PDD'

/content/drive/MyDrive/dev/aiac/AIAC-PDD


In [3]:
# !pip install -U PyYAML
# !pip install Pillow==9.0.0
# !pip install toml

## Uncomment and run codes below if this is the first time running this notebook

In [3]:
# !pip install -qr dspYolov5/requirements.txt  # install dependencies for yolov5

### uncomment codes below to download training data

In [11]:
# Uncomment the lines below to download training data
# !gdown 'https://drive.google.com/uc?id=1WIW0gPc8LFDbysI2gycfpnlXRMDZkoIs'
# !unzip 'td.zip' -d './cvat'

In [12]:
# cvat folder should contain 3 folders: td1, td2, td3, td4
!ls cvat 

__MACOSX  td1  td2  td3  td4


## Uncomment block of codes in cell below to convert cvat annotations to yolo annotations

In [4]:
# import os
# from dsps.simple_fxn import cvat2yolo
# # location of cvat annotations
# annotation_folder = 'cvat/'
# output_path = 'data/training'
# val_path = 'data/validation'
# for annts in os.listdir(annotation_folder):
#   json_file = os.path.join(annotation_folder,annts,'annotations','instances_default.json')
#   img_path = os.path.join(annotation_folder,annts,'images')
#   if(os.path.isfile(json_file)):
#     cvat2yolo(annts,json_file,img_path, output_path, val_path)

## Train a new model

### If not running on GPU, change runtime to GPU
________

In [5]:
import json
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu113 (Tesla P100-PCIE-16GB)


### **Change class labels and number of classes**: 
* Edit dspYolov5/data/dsps.yaml file 
### **Change model configuration**: 
* Edit dspYolov5/models/dsps_model.yaml

### Install albumentations for augmentation. You may need to restart kernel

In [6]:
# !pip install -U albumentations
# !pip install "opencv-python-headless<4.3"

### run the code below. You may get an error the first time. 

---

restart the runtime and run again. 

stop cell after total epoch have elapsed. 

In [ ]:
!python -i dspYolov5/train_dsps.py --data dspYolov5/data/dsps.yaml --cfg dspYolov5/models/dsps_model.yaml --batch-size 8 --device 0

train: weights=yolov5s.pt, cfg=dspYolov5/models/dsps_model.yaml, data=dspYolov5/data/dsps.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=150, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=weights/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
fatal: ambiguous argument 'main_dev..origin/master': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'
github: Command 'git rev-list main_dev..origin/master --count' returned non-zero exit status 128.
YOLOv5 🚀 418ac63 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

hyperparame

### Evaluate and Visualize Model Performance

#### change weights path in the following cell: replace exp12 in the weights path with your corresponding folder-- 'weights/train/exp12/weights/best.pt'

In [8]:
## Uncomment the lines below to download test data
# !rm -r 'data/test_images'
# !gdown 'https://drive.google.com/uc?id=1ihHwj1AHHuHGnkbVAowXC6b0y7zHqWEe'
# !unzip 'ts_img.zip' -d './data/test_images'

In [ ]:
## create results folder if it doen't exist

In [9]:
# !python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp/weights/best.pt' --img 640 --conf 0.25 --augment --source data/test_images/images
!python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp2/weights/best.pt' --img 640 --conf 0.4 --iou-thres 0.999 --augment --source data/test_images/ts

100% 755k/755k [00:00<00:00, 48.3MB/s]
detect: weights=['weights/train/exp2/weights/best.pt'], source=data/test_images/ts, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.999, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /content/drive/MyDrive/dev/aiac/AIAC-PDD/requirements.txt not found, check failed.
YOLOv5 🚀 418ac63 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Fusing layers... 
Model Summary: 224 layers, 7070092 parameters, 0 gradients
/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwar

### Upload submission.json file in the "results" folder to leaderboard

In [ ]:
## sample submission file content
# [{"1 (134).JPG": [[3, 365, 585, 895, 697], [2, 41, 588, 917, 2000]]}, {"1 (154).JPG": [[1, 8, 1622, 429, 2007], [6, 24, 0, 157, 867]]}]
# [3, 365, 585, 895, 697] --> ['class','x1','y1','x2','y2']
# x1,y1 - top left corner of bounding box
# x2,y2 _ bottom right corner of bounding box